<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=4>Módulo 8</font></h2>



<h1><font color="#00586D" size=5>Sistema de recomendación de libros</font></h1>

<br><br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Luis de la Ossa</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Introducción](#section1)
   * [Lectura de las revisiones](#section11)
   * [Etiquetas](#section12)
   * [Preparación de los datos](#section13)
* [2. Limpieza del texto](#section2)
* [3. Búsqueda por similaridad de la representación tf-idf](#section3)
* [4. Recomendación basada en contenido](#section4)
   * [Selección de libros preferidos por el usuario](#section41)
   * [Búsqueda de libros similares](#section42)
   * [Priorización de resultados y recomendación](#section43)
* [5. Sistema híbrido de recomendación](#section5)  
* [6. LDA y búsqueda por similaridad](#section6)
<br>

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

<a id="section1"></a>
## <font color="#00586D"> 1. Introducción</font>
<br>

Los sistemas de recomendación basados en filtrado colaborativo utilizan de manera exclusiva los perfiles de votación de usuarios/items, y no consideran el contenido. Como se vió en clase,  permiten obtener resultados que, si bien cualitativamente son aceptables, pueden llegar a desconcertar al no corresponderse con lo esperado. 

En este proyecto se diseñará un pequeño sistema de recomendación de libros basado en contenido. Para ello, será necesario hacer uso de algunos de los conceptos relaccionados con aprendizaje automático sobre información textual (_Text Mining_) que han sido tratados también a lo largo del módulo. 

Como punto de partida, se partirá del conjunto de datos [goodbooks-10k](https://www.kaggle.com/zygmunt/goodbooks-10k) disponible en [kaggle](https://www.kaggle.com). Éste contiene información relativa a 10000 libros obtenida de la red social  [goodreads](http://goodreads.com), que actualmente es el sitio de referencia en la red para aficcionados a la lectura. Además de títulos y autores, el conjunto de datos incluye votos y etiquetas aportadas por más de 53000 usuarios. 

<div class="alert alert-block alert-warning">
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Se han hecho algunas modificaciones con respecto a la base original para que sea menos tedioso manejar los distintos índices e identificadores. 
</div>

In [2]:
import pandas as pd
import numpy as np

df_goodreads = pd.read_csv('data/books.csv', sep="\t", index_col=0)
df_goodreads.head(2)

,gr_book_id,gr_best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
book_id,,,,,,,,,,,,,,,,,,,,,
0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section11"></a> 
###  <font color="#00586D"> Lectura de las revisiones </font>
<br>

Los datos de la tabla apenas contienen información relativa al contenido de cada libro. Sin embargo, es posible acceder a los resúmenes almacenados en la propia web [GoodReads](www.goodreads.com) que, en cierto grado, aportan esta información. En el [anexo](#Anexo) se adjunta la función utilizada para obtenerlos. 

El resumen de cada libro ha sido almacenado en un archivo de texto denominado `./data/overviews/bookid.txt`, donde _bookid_ corresponde al identificador del libro (columna `book_id` de `df_goodreads`). 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Implementar una función, denominada `get_overview(gr_book_id)`, que reciba el identificador de un libro, lea el archivo de texto que contiene el resumen correspondiente y lo devuelva en un _String_ (o devuelva `None` si este resumen no existe). 

In [3]:
def get_overview(gr_book_id):
    try:
        with open(f'data/overviews/{gr_book_id}.txt', "r") as archivo:
            contenido = archivo.read()
        return contenido
    except:
        return None
    
get_overview(320) # Cien años de soledad (Gabriel García Márquez)

'(Book Jacket Status: Jacketed)The brilliant, bestselling, landmark novel that tells the story of the Buendia family, and chronicles the irreconcilable conflict between the desire for solitude and the need for loveâ€”in rich, imaginative prose that has come to define an entire genre known as "magical realism."'

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true"></i></font> Crear una columna en `df_goodreads`, denominada `overview`, que contenga la revisión del libro correspondiente. Rellenar los valores vacíos de esa columna con un _String_ de longitud cero ("").

In [4]:
df_goodreads['overview'] = df_goodreads['gr_book_id'].apply(lambda x: get_overview(x)).fillna('')

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_, aunque no hay que llamar a una lambda. Con esto vale. 
```python
df_goodreads['gr_book_id'].apply(get_overview)
```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section12"></a> 
###  <font color="#00586D">Etiquetas </font>
<br>


En el conjunto de datos se proporcionan etiquetas relativas a cada libro que han sido aportadas por los usuarios. Esta información está incluída en dos archivos. El primero de ellos, `tags.csv`, contiene el identificador de cada etiqueta y el código correspondiente. El segundo, `book_tags.csv` contiene las etiquetas relativas a cada libro. Se almacenarán, respectivamente, en los _DataFrame_ `df_tags` y `df_book_tags`.

In [5]:
df_tags = pd.read_csv('./data/tags.csv')
df_book_tags = pd.read_csv('./data/book_tags.csv')

Etiquetas

In [6]:
df_tags.iloc[2000:2002]

,tag_id,tag_name
2000,2000,alex-read
2001,2001,alex-rider


Etiquetas por libro

In [7]:
df_book_tags.head(3)

,gr_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Incorporar el campo `tag_name` de `df_tags` a `df_book_tags`, sustituyendo el campo `tag_id` por la etiqueta (_String_) correspondiente. Para ello, fundir los dos _DataFrame_ y, posteriormente, eliminar las columnas `count` y `tag_id`.

In [8]:
df_book_tags = (df_book_tags
                .merge(df_tags,
                       left_on='tag_id',
                       right_on='tag_id')
                .drop(columns=['tag_id', 'count']))

df_book_tags.tail()

,gr_book_id,tag_name
999907,31538635,hogwarts
999908,32848471,jan-2017
999909,33288638,single-mom
999910,33288638,fave-author
999911,33288638,slowburn


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Inspeccionar las etiquetas que aparecen en `df_book_tags` y el número de veces que aparece cada una. ¿Deberían eliminarse algunas?

In [9]:
df_book_tags['tag_name'].unique().shape

(34252,)

In [10]:
df_book_tags.groupby('tag_name').count().sort_values(by='gr_book_id').head(50)

,gr_book_id
tag_name,
ｆａｖｏｕｒｉｔｅｓ,1
queer-fantasy,1
forensic-medicine-thriller,1
queen-elizabeth-ii,1
queen-elizabeth,1
forensic-sculptor,1
quebec-city,1
quarterbacks,1
quanah-parker,1


Existe un gran numero de etiquetas y muchas de ellas aparecen una vez. Para facilitar el analisis podrian eliminarse en base a algun criterio.

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Una de las cosas que se observan es que hay un alto número de etiquetas que aparecen una vez, y que son irrelevantes, por lo que es mejor ignorarlas. 


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Eliminar todas las etiquetas que aparezcan menos de 20 veces. 

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i> Este ejercicio se puede hacer de varias formas. Una de ellas, consiste en agrupar por `tag_name`, mediante `groupby`, y filtrar con `filter` los grupos con tamaño >=20.
</div>

In [11]:
df_book_tags = df_book_tags.groupby('tag_name').filter(lambda x: len(x) >= 20)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Otro de los problemas que se aprecian es que algunas etiquetas son genéricas, y no corresponden a libros concretos. Por ejemplo palabras como `read-readings` o `favourites`. 

In [12]:
df_book_tags['tag_name'].value_counts().nlargest(25)

to-read              9983
favorites            9881
owned                9858
books-i-own          9799
currently-reading    9776
library              9415
owned-books          9221
fiction              9097
to-buy               8692
kindle               8316
default              8239
ebook                8054
my-books             7561
audiobook            7242
ebooks               7203
wish-list            7192
my-library           7000
audiobooks           6862
i-own                6670
adult                6604
audio                6548
favourites           6422
novels               5665
own-it               5514
contemporary         5287
Name: tag_name, dtype: int64

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Eliminar todas las etiquetas que contengan estos términos (los términos de la lista `target_tags`).  

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
En este caso, es recomendable no utilizar las etiquetas completas para eliminar así sus variantes también. Por otra parte, `Series.str.contains` acepta expresiones regulares.
</div>

In [13]:
target_tags = ['read', 'favo', 'own', 'top', 'book', 'librar', 'kindle', 'list']

df_book_tags = df_book_tags[~df_book_tags['tag_name'].str.contains('|'.join(target_tags), case=False, regex=True)]
df_book_tags

,gr_book_id,tag_name
9983,1,fantasy
9984,2,fantasy
9985,3,fantasy
9986,5,fantasy
9987,6,fantasy
...,...,...
997632,29430013,best-of-2016
997633,29519514,best-of-2016
997634,30008702,best-of-2016
997635,30364931,best-of-2016


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Por último,  se creará un `DataFrame` denominado `df_book_tag_text` en el que, para cada libro (indicado por su código `goodreads_book_id`), se añadirá una columna con _un solo campo de texto_, resultado de unir las etiquetas correspondientes.  

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Agrupar las entradas `DataFrame` en `df_book_tags` en función del campo `goodreads_book_id` y unir todas las etiquetas de cada grupo mediante `join`. Almacenar el resultado en un `DataFrame` denominado `df_book_tag_text`.

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Para pasos posteriores es necesario convertir el resultado resultante, una `Serie`, en un `DataFrame`, mediante `Series.to_frame()`. 
</div>

In [14]:
df_book_tag_text = df_book_tags.groupby('gr_book_id')['tag_name'].apply(lambda x: ' '.join(x)).to_frame()

df_book_tag_text.head()

,tag_name
gr_book_id,
1,fantasy young-adult fiction harry-potter ya se...
2,fantasy young-adult fiction harry-potter ya ma...
3,fantasy young-adult fiction harry-potter ya se...
5,fantasy young-adult fiction harry-potter ya se...
6,fantasy young-adult fiction harry-potter ya se...


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Renombrar la columna `tag_name` de `df_book_tags` a `text_tags`. Rellenar los valores perdidos con un String vacío ("").

In [15]:
df_book_tag_text = df_book_tag_text.rename(columns={'tag_name': 'text_tags'}).fillna('')

df_book_tag_text.head()

,text_tags
gr_book_id,
1,fantasy young-adult fiction harry-potter ya se...
2,fantasy young-adult fiction harry-potter ya ma...
3,fantasy young-adult fiction harry-potter ya se...
5,fantasy young-adult fiction harry-potter ya se...
6,fantasy young-adult fiction harry-potter ya se...


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Incorporar la información del texto de las etiquetas, es decir, la columna `text_tags`, al `DataFrame` principal `df_goodreads` (utilizar _merge_).

In [16]:
df_goodreads = pd.merge(df_goodreads, df_book_tag_text, left_on='gr_book_id', right_index=True, how='inner')

df_goodreads.head(2)

,gr_book_id,gr_best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,overview,text_tags
book_id,,,,,,,,,,,,,,,,,,,,,
0,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",...,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...,fantasy young-adult fiction ya series adventur...
1,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...,fantasy young-adult fiction harry-potter ya se...


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<a id="section2"></a> 
## <font color="#00586D"> 2. Limpieza y preparación del texto </font>
<br>

En este proyecto, el contenido (texto) asociado a cada libro se representa mediante un modelo de bolsa de palabras. Como se ha explicado a lo largo del módulo, cuando se trabaja con este tipo de representación es recomendable limpiar el texto y eliminar información irrelevante. Para ello se utilizará la librería `spacy`.  En primer lugar, se utilizará el modelo de lenguaje `en_core_web_sm`. Aunque no es el modelo con mejor rendimiento, es suficiente para este contexto, y es más eficiente que `en_core_web_trf`(basado en *transformers*, y que es el que mejor funciona).

In [17]:
#! python -m spacy download es_dep_news_trf
#! python -m spacy download en_core_web_sm # Solo se utilizará este
#! python -m spacy download en_core_web_trf 

In [18]:
import spacy 

# Crea un objeto con el pipeline
nlp = spacy.load("en_core_web_sm")
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Crear una función denominada `clean` que acepte un texto, lo convierta en un documento (`spacy`) y devuelva un *String* compuesto por los lemas correspondientes a cada token, en minúscula, y descartando los tokens que no sean alfanuméricos, o los que correspondan a *stopwords*.

In [19]:
def clean(overview):
    doc = nlp(overview)
    cleaned_text = " ".join([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop])
    return cleaned_text
    
overview = df_goodreads.iloc[0]['overview']
clean(overview)

'win famous lose mean certain death nation panem form post apocalyptic north america country consist wealthy capitol region surround poor district early history rebellion lead district capitol result destruction creation annual televise event know hunger games punishment reminder power grace capitol district yield boy girl age lottery system participate game tribute choose annual reaping force fight death leave survivor claim victory year old katniss young sister prim select district female representative katniss volunteer place male counterpart peeta pit big strong representative train life see death sentence katniss close death survival second nature'

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font>Aplicar la función `clean` sobre cada texto almacenado en la columna `df_goodreads['overview']`para preprocesarlo, almacenando el resultado en la propia columna.


<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Este ejercicio tarda unos cinco minutos en ejecutarse. Se ha planteado así (sin utilizar `nlp.pipe`) porque es más sencillo, y en este caso tarda prácticamente lo mismo.
</div>

In [20]:
df_goodreads['overview'] = df_goodreads['overview'].apply(lambda x: clean(x))

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_. Igual que antes. No hace falta el lambda. 
```python

```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section13"></a> 
###  <font color="#00586D"> Preparación de datos </font>
<br>

Una vez incorporada la información relativa a resúmenes y etiquetas, y puesto que el texto se tratará simplemente como una bolsa de palabras, esta se fundirá, constituyendo el contenido de cada libro. 


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i></font> 
Fundir las columnas `overview` y `text_tags` de `df_goodreads` en otra columna llamada`text`. Eliminar las columnas en las que `text` tenga longitud 0.

In [21]:
df_goodreads['text'] = df_goodreads['overview'] + ' ' + df_goodreads['text_tags']
df_goodreads = df_goodreads[df_goodreads['text'].str.len() > 0]

df_goodreads.shape

(10000, 25)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


In [22]:
data_books = df_goodreads['text']

####  <font color="#00586D"> Selección de cinco libros para pruebas </font>


A continuación, se eligen cinco libros, que serán almacenados en el `DataFrame` `df_books_test`

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Como no vamos a hacer predicción, no pasa nada porque los libros estén en dos sitios. 
</div>

In [23]:
test_pos = np.array([2,4,7,9,12]) # Sugerencia. Podéis cambiarlos. 

df_books_test = df_goodreads.iloc[test_pos].copy()
df_books_test

,gr_book_id,gr_best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,...,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,overview,text_tags,text
book_id,,,,,,,,,,,,,,,,,,,,,
2,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",...,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,thing absolutely positive edward vampire secon...,fantasy young-adult fiction ya sci-fi-fantasy ...,thing absolutely positive edward vampire secon...
4,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,...,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,publication great gatsby largely dismiss light...,young-adult fiction default classics 5-stars n...,publication great gatsby largely dismiss light...
7,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,The Catcher in the Rye,...,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...,hero narrator catcher rye ancient child sixtee...,young-adult fiction ya default classics novels...,hero narrator catcher rye ancient child sixtee...
9,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,Pride and Prejudice,...,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...,truth universally acknowledge single man posse...,young-adult fiction default classics 5-stars n...,truth universally acknowledge single man posse...
12,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,1984,...,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...,seminal text century nineteen eighty rare work...,fantasy young-adult fiction sci-fi-fantasy def...,seminal text century nineteen eighty rare work...


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section3"></a> 
## <font color="#00586D"> 3. Búsqueda por similaridad de la representación tf-idf </font>
<br>

La información _tf-idf_ es muy útil de cara a clasificar y comparar documentos. Cada texto se representa  mediante un vector de valores _tf-idf_, y la búsqueda de documentos se apoya en una medida de similaridad, la _similaridad coseno_, para esta representación.

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i></font> Crear un objeto `TfidfVectorizer` de `sklearn`, denominado `tfidf_vect`, que represente un máximo de 20000 términos. Obtener la matriz _tf-idf_ de los datos almacenados en `data_books` y almacenarla en una variable denominada `data_books_tfidf`. Extraer los términos considerados en `tfidf_vect`, y almacenarlos en la variable `terms`.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_features=20000)

data_books_tfidf = tfidf_vect.fit_transform(data_books)

terms = tfidf_vect.vocabulary_

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_. Es mejor hacer la última parte así:
```python
terms = tfidf_vect.get_feature_names_out()
```
</div>


#### <font color="#00586D"> Búsqueda del libro más similar </font>

Se tomará un libro incluído en el conjunto de datos `df_books_test`. Por ejemplo, el primero.

In [25]:
print(df_books_test.iloc[0])
text_query = df_books_test.iloc[0].text
text_query

gr_book_id                                                               41865
gr_best_book_id                                                          41865
work_id                                                                3212258
books_count                                                                226
isbn                                                                 316015849
isbn13                                                         9780316015840.0
authors                                                        Stephenie Meyer
original_publication_year                                               2005.0
original_title                                                        Twilight
title                                                  Twilight (Twilight, #1)
language_code                                                            en-US
average_rating                                                            3.57
ratings_count                                       

'thing absolutely positive edward vampire second know dominant thirst blood unconditionally irrevocably love book twilight saga internationally bestselle author stephenie meyer introduce bella swan edward cullen pair star cross lover forbidden relationship ripen backdrop small town suspicion mysterious coven vampire love story bite fantasy young-adult fiction ya sci-fi-fantasy default 5-stars novels fantasy-sci-fi paranormal ya-fantasy teen english urban-fantasy supernatural ya-fiction young-adult-fiction scifi-fantasy faves contemporary on-my-shelf have movie finished sci-fi ya-lit werewolves séries love science-fiction finished-series youngadult abandoned did-not-finish dnf american movies drama high-school romantic horror chick-lit first-in-series teen-fiction meh young-adults guilty-pleasures guilty-pleasure paranormal-romance ya-paranormal ya-romance vampires vampire completed-series never-again vamps paranormal-fantasy fantasy-paranormal pnr love-triangle'

Para poder buscar otro libro por similaridad, es necesario transformar el texto de consulta a la representación _tf-idf_. Previamente, ha de ser preprocesado (limpiado con la función `clean`).

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Transformar el documento `text_query` a formato _tf-idf_ y almacenar el resultado en `text_query_tfidf`.

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
Esta sería la manera de proceder con libros nuevos. No obstante, en este caso concreto, ___como el libro ya estaba en el corpus y se ha obtenido su representación___, se podría obtener el vector directamente desde `data_books_tfidf`.

</div>

In [26]:
text_query_tfidf = tfidf_vect.transform([clean(text_query)])

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Mostrar los 10 términos más relevantes (con mayor valor _tf-idf_) para el documento `text_query`.

In [27]:
top_indices = text_query_tfidf.toarray().argsort()[0][-10:][::-1]

top_terms = np.array(list(terms.keys()))[top_indices]

print(top_terms)

['untethered' 'pagford' 'ichijo' 'implode' 'homage' 'omnipresent' 'aysel'
 'embezzle' 'laughable' 'whomever']


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. Este está mal. Sería así, siempre y cuando hayas obtenido bien el vocabulario. 
```python
" ".join([terms[id_term] for id_term in text_query_tfidf.toarray()[0].argsort()[:-11:-1]])
```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Calcular las similaridades (similaridad coseno) del documento `text_query_tfidf` con el resto de documentos en la matriz `data_books_tfidf`, y almacenarlas en un vector denominado `similarities_tfidf`.

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
La función `cosine_similarity` toma como argumentos dos matrices y devuelve una matriz (aunque sea unidimensional). Para obtener un vector unidemensional, y operar posteriormente con más agilidad, `similarities_tfidf` ha de almacenar el elemento cero de la matriz devuelta por la función, o transformar esta matriz en un vector. 
</div>

In [28]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities_tfidf = cosine_similarity(text_query_tfidf, data_books_tfidf)[0]

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Obtener los índices de los 6 libros más similares a `text_query` (obviamente, el más similar será el propio libro).



In [29]:
top_similarities = similarities_tfidf.argsort()[:-7:-1]

df_goodreads.iloc[top_similarities]['title']

book_id
2                                 Twilight (Twilight, #1)
48                                New Moon (Twilight, #2)
51                                 Eclipse (Twilight, #3)
4820    Jessica's Guide to Dating on the Dark Side (Je...
991                    The Twilight Saga (Twilight, #1-4)
1539                         Masquerade (Blue Bloods, #2)
Name: title, dtype: object

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Se aprecia que, efectivamente, los libros son muy similares. De hecho, todos corresponden a la saga. Esto puede deberse al uso de palabras clave muy concretas como _"Twilight"_.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section4"></a> 
## <font color="#00586D"> 4. Recomendación basada en contenido </font>
<br>

Existen muchas posibilidades en la implementación de sistemas de recomendación basados en contenido. En general, todas requieren:

* Determinar qué ítems prefiere el usuario
* Encontrar ítems similares
* Priorizar esos ítems


Para ilustrar el funcionamiento de este tipo de sistemas, se tomarán los datos relativos a la actividad de un usuario escogido al azar, y se devolverá un conjunto de libros recomendados. 


El archivo `'./data/ratings.csv` contiene las valoraciones hechas por más de 53000 usuarios a los 10000 libros. En total, contiene cerca de un millón de entradas en formato (`user_id`, `book_id`, `rating`). El tamaño de la base de datos dificulta el trabajo con una matriz, aunque sea dispersa, ya que el proceso de elaboración (mediante `pivot` es muy lento). 

In [30]:
df_ratings = pd.read_csv('./data/ratings.csv', sep=',')

display(df_ratings.head())
df_ratings.shape

,user_id,book_id,rating
0,313,0,5
1,438,0,3
2,587,0,5
3,1168,0,4
4,1184,0,4


(981756, 3)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>


---

<a id="section41"></a> 
### <font color="#00586D"> Selección de libros preferidos por el usuario </font>
<br>

El primer paso de la recomendación consiste en determinar qué libros prefiere el usuario.  Una posibilidad consiste en seleccionar aquellos para los que éste ha otorgado una puntuación mayor que tres. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Tomar un usuario al azar y devolver las entradas correspondientes a los libros que ha votado. Almacenarlas en el `DataFrame` `ratings_user`. Descartar de `ratings_user` todos los libros con puntuación menor que cuatro. 

In [31]:
np.random.seed(0)
test_user = np.random.randint(max(df_ratings['user_id']))

ratings_user = df_ratings[(df_ratings['user_id']==test_user) & (df_ratings['rating'] >= 4)]
ratings_user

,user_id,book_id,rating
53101,2732,531,5
209776,2732,2099,5
214080,2732,2142,5
242344,2732,2425,4
369792,2732,3702,4
401262,2732,4018,4
425738,2732,4264,5
476351,2732,4773,4
524632,2732,5262,5
543609,2732,5454,4


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section42"></a> 
### <font color="#00586D"> Búsqueda de libros similares </font>
<br>

Una vez se dispone de la lista de libros, es necesario encontrar libros similares. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font>Extraer la representación de los libros relativos al usuario y copiarla en una matriz denominada `user_books_tfidf`. 

In [32]:
books_user_id = ratings_user['book_id'].to_list()

user_books_tfidf = data_books_tfidf.toarray()[books_user_id]

user_books_tfidf.shape

(14, 20000)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Calcular la matriz de similaridades coseno entre `user_books_tfidf`, `data_books_tfidf`. Fijar las similaridades de los libros leídos con el usuario  (`books_user_id `) consigo mismos a -1 con el fin de descartarlos. 

In [33]:
similarities_user = cosine_similarity(user_books_tfidf, data_books_tfidf)
similarities_user[:,books_user_id]=-1

similarities_user.shape

(14, 10000)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener los 3 libros más similares _a cada uno_ de los leídos por el usuario y almacenarlos en la matriz `similar_books`.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> La función `similarities_user.argsort(axis=1)` devuelve los índices ordenados (ascendentemente) para cada fila del array. Se trata de coger las 3 últimas columnas de `similarities_user`. 
    
Por otra parte, el resultado de este ejercicio ha de ser un array ***unidimensional*** sin elementos repetidos.
</div>

In [34]:
similar_books = np.unique(similarities_user.argsort(axis=1)[:,-3:])

print(similar_books, "("+str(len(similar_books))+" libros distintos.)")

[  77  156  175  304  641  865 1074 1311 1487 1761 1882 2109 2474 2513
 3027 3292 3565 3848 3885 4172 5057 6190 6335 6444 6706 6715 6889 7087
 7454 8249 8419 8624 9600 9697 9771 9778] (36 libros distintos.)


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section43"></a> 
### <font color="#00586D"> Priorización de resultados y recomendación</font>

Llegados a este punto, se han obtenido los libros más similares a los leídos y valorados positivamente por el usuario. Una posibilidad a la hora de priorizar los resultados consistiría en dar más peso a aquellos libros similares que han aparecido más veces en la matriz `similar_books`. 

También es posible utilizar la información del `DataFrame` `df_goodreads` para priorizar los libros. En concreto, la columna `average_rating` contiene la valoración media de cada libro en la plataforma, y puede ser utilizada para priorizar. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener los 10 libros con mayor `average_rating` de entre los más similares a los valorados positivamente por el usuario (`similar_books`). Almacenar sus índices en un array denominado `rec_books_user_id`.

In [35]:
rec_books_user_id = (df_goodreads
                     .loc[similar_books
                          .tolist()]
                     .sort_values(by='average_rating',
                                  ascending=False)
                     .head(10)
                     .index.values)
rec_books_user_id

array([6444,  156, 1882, 6889, 2109,  641,  175, 9697, 7454, 9600],
      dtype=int64)

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Por último, se imprimen los nombres de los libros preferidos por el usuario, cuyos índices están almacenados en `books_user_id`, y la recomendación, cuyos índices están almacenados en `rec_books_user_id`.

In [36]:
print("Libros preferidos por el usuario: ")
print("----------------------------------")
for book in df_goodreads.iloc[books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)
    
print("\n\nRecomendación: ")
print("----------------------------------")
for book in df_goodreads.iloc[rec_books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)   

Libros preferidos por el usuario: 
----------------------------------
Frank Miller, David Mazzucchelli, Richmond Lewis, Dennis O'Neil 
	 Batman: Year One
Frank Miller 
	 Batman: The Dark Knight Returns #1
Mark Waid, Alex Ross, Elliot S. Maggin 
	 Kingdom Come
Frank Miller, Lynn Varley 
	 300
Philip Roth 
	 The Human Stain (The American Trilogy, #3)
Kurt Busiek, Alex Ross 
	 Marvels
A. Manette Ansay 
	 Vinegar Hill
Michael Blake 
	 Dances with Wolves (Dances with Wolves, #1)
Alexander McCall Smith 
	 The Full Cupboard of Life (No. 1 Ladies' Detective Agency, #5)
Gary Paulsen 
	 Brian's Winter (Brian's Saga, #3)
Richard Russo 
	 Bridge of Sighs
Tracy Chevalier 
	 Falling Angels
Stephen King 
	 Apt Pupil
Dr. Seuss 
	 And to Think That I Saw it on Mulberry Street


Recomendación: 
----------------------------------
Katsuhiro Otomo, Yoko Umezawa, Linda M. York, Jo Duffy 
	 Akira, Vol. 1
Dr. Seuss, לאה נאור 
	 Green Eggs and Ham
John  Williams, John McGahern 
	 Stoner
Scott Snyder, Greg Capu

Puede observarse que, efectivamente, los títulos recomendados guardan mucha relación con los títulos valorados positivamente por el usuario. 

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section5"></a> 
##  <font color="#00586D"> 5. Sistema híbrido de recomendación  </font>
<br>

<br> En el apartado anterior se ha utilizado la valoración media de cada libro para priorizar los vecinos más cercanos. Existe otra posibilidad, que consite en utilizar las valoraciones de los libros obtenidas mediante filtrado colaborativo. Esto constituye un sistema híbrido, ya que se mezclan las dos aproximaciones. Por una parte, se seleccionan libros basados en contenido, y de ellos, se muestra el conjunto para el que se predice una mayor puntuación por parte del usuario.
<br>

No existe mucho material disponible en relación a este tipo de modelos. Tampoco una librería de referencia, aunque va cobrando popularidad [surprise](http://surpriselib.com/). En primer lugar, se utilizará esta librería para obtener los scores mediante filtrado colaborativo. En concreto, se utilizará el algoritmo SVD (visto en clase). 

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> El autor principal de la librería describe en en una serie de artículos [enlace](http://nicolas-hug.com/blog/) como funciona la versión básica de este algoritmo. La lectura es muy recomendable. 
</div>

En la siguiente celda, se cargan los votos  en la estructura `ratings_data` (usada por `surprise`) y se aprende un modelo SVD.

In [37]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection.split import train_test_split

reader = Reader()
ratings_data = Dataset.load_from_df(df_ratings, reader).build_full_trainset()
svd = SVD(n_factors=10)
svd.fit(ratings_data);

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i> </font> Obtener uno a uno, las valoraciones de los libros similares a los preferidos por el usuario (`similar_books`), y tomar los 10 con más valoración. 

In [38]:
scores = np.zeros(similar_books.shape)

for i, book_id in enumerate(similar_books):
    score = svd.predict(test_user, book_id).est
    scores[i] = score
    
rec_books_user_id = similar_books[scores.argsort()[-11:-1].tolist()]

print("\n\nRecomendación: ")
print("----------------------------------")
for book in df_goodreads.iloc[rec_books_user_id][['title', 'authors']].itertuples():
    print(book.authors, "\n\t",book.title)    



Recomendación: 
----------------------------------
Warren Ellis, Adi Granov 
	 Iron Man: Extremis
Scott Snyder, Greg Capullo, Jonathan Glapion, James Tynion IV, Rafael Albuquerque, Jason Fabok, Becky Cloonan, Andy Clarke, Sandu Florea 
	 Batman, Volume 2: The City of Owls
Jean Craighead George 
	 My Side of the Mountain (Mountain, #1)
Ken Follett 
	 World Without End (The Kingsbridge Series, #2)
Bryan Lee O'Malley 
	 Scott Pilgrim, Volume 2: Scott Pilgrim vs. The World
Alan Moore, Curt Swan, George Pérez, Kurt Schaffenberger 
	 Superman: Whatever Happened to the Man of Tomorrow?
Dr. Seuss, לאה נאור 
	 Green Eggs and Ham
Craig Thompson 
	 Habibi
Katsuhiro Otomo, Yoko Umezawa, Linda M. York, Jo Duffy 
	 Akira, Vol. 1
John  Williams, John McGahern 
	 Stoner


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>


---

<a id="section6"></a> 
## <font color="#00586D"> 6. LDA y búsqueda por similaridad </font>
<br>

LDA permite modelar los grupos que dan lugar al corpus de documentos, de modo que cada documento se representa como una mixtura de las distribuciones de probabilidad de cada grupo, dada por un vector de pesos. Esta representación puede ser utilizada también para caracterizar el documento (en lugar de _tf-idf_) y permite cuantificar la similaridad con otros. Por tanto, también serviría para implementar un sistema de recomendación (basado en contenido o híbrido) similar al anterior.

LDA trabaja con la matriz de frecuencias de  términos por documento (_term frecuency_). Por tanto, es necesario primero obtenerla. 

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Construir un objeto `CountVectorizer` de `sklearn` y almacenarlo en la variable `cv_vect`.  Obtener la matriz `data_books_cv` a partir de los datos almacenados en `data_books`. Utilizar un tamaño máximo de vocabulario de 5000 (para que LDA no requiera demasiado tiempo y memoria). Almacenar el vocabulario generado en `cv_vect` en la variable `terms`. 

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

cv_vect = CountVectorizer(max_features=5000)

data_books_cv = cv_vect.fit_transform(data_books)

terms = cv_vect.get_feature_names_out()

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


Una vez construida la matriz de frecuencias, se ejecutará el algoritmo LDA, para representar los documentos mediante 20 grupos.

In [40]:
from sklearn.decomposition import LatentDirichletAllocation

# Número de temas
n_topics = 20

lda = LatentDirichletAllocation(n_components=n_topics, max_iter=20, learning_method='online',
                                learning_offset=50., random_state=0, n_jobs=-1)

 <font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Construir el modelo LDA y almacenarlo en la variable `topic_term`. Guardar la representación de los documentos en la variable `data_books_lda`.

In [41]:
from time import time
start = time()

data_books_lda = lda.fit_transform(data_books_cv)
topic_term = lda.components_

print("Tiempo: {:0.3f}s.".format(time() - start))
print("Tamaño del modelo:", topic_term.shape)
print("Tamaño del resultado de la transformación: ", data_books_lda.shape)

Tiempo: 105.746s.
Tamaño del modelo: (20, 5000)
Tamaño del resultado de la transformación:  (10000, 20)


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


La siguiente función (de la práctica de LDA) imprime los términos y scores más importantes para cada grupo.

In [42]:
def topic_relevant_words(topic_id, model, terms, num_words):
    print("Grupo: ", topic_id)
    print("----------------------")
    # Extrae las posiciones de los mayores scores. 
    top_term_ids = model[topic_id,:].argsort()[:-num_words-1:-1]
    # Extrae los términos de las posiciones correspondientes
    top_terms = [terms[id_term] for id_term in top_term_ids]
    # Extrae y calcula los scores
    top_scores =  model[topic_id,top_term_ids] / np.sum(model[topic_id,:])
    # Imprime los resultados
    for term, score in zip(top_terms,top_scores):
        print("{:s}:     \t{:.4f}".format(term,score))
    print("----------------------\n")

topic_relevant_words(0,topic_term, terms, 10)

Grupo:  0
----------------------
children:     	0.0753
fiction:     	0.0434
childrens:     	0.0389
childhood:     	0.0296
lit:     	0.0294
young:     	0.0253
middle:     	0.0240
kids:     	0.0218
ya:     	0.0213
school:     	0.0211
----------------------



#### <font color="#00586D"> Búsqueda del libro más similar </font>

<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font>  Obtener la representación a partir del modelo `lda` del documento `text_query`. Almacenarla en la variable `text_query_lda`. 

In [43]:
text_query_lda = lda.transform(cv_vect.transform([text_query]))
text_query_lda

array([[3.78787885e-04, 3.78787879e-04, 3.78787882e-04, 3.78787883e-04,
        3.74018802e-01, 3.78787879e-04, 3.78787894e-04, 3.78787884e-04,
        8.08293164e-02, 3.78787888e-04, 3.78787879e-04, 8.79770315e-02,
        3.78787886e-04, 4.04918794e-01, 3.78787879e-04, 4.65742379e-02,
        3.78787883e-04, 3.78787884e-04, 3.78787879e-04, 3.78787881e-04]])

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Mostrar las características de los tres grupos más relevantes en el texto `text_query`. 

In [44]:
top_topics = text_query_lda.argsort()[:, -3:]
top_topics = top_topics[0][::-1]
for topic in top_topics:
    print("Tema {}: {}".format(topic, lda.components_[topic]))

Tema 13: [0.05000001 0.05000001 0.05       ... 0.05       0.05       0.05      ]
Tema 4: [0.05 0.05 0.05 ... 0.05 0.05 0.05]
Tema 11: [0.05000001 0.05000006 0.05000001 ... 0.05       0.05       0.05      ]


<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. Aquí se pedía utilizar topic_relevant_words. Si no, no se ve nada.
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Calcular las similaridades (distancia_euclídea) del documento `text_query_lda` con el resto de documentos en la matriz `data_books_lda`, y almacenarlas en un vector denominado `similarities_lda`.

In [45]:
from sklearn.metrics.pairwise import euclidean_distances

similarities_lda = euclidean_distances(text_query_lda, data_books_lda)[0]

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


<font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#00586D"></i></font> Obtener los índices de los 10 libros más similares a `text_query`. Hay que tener en cuenta que `euclidean_distances` devuelve distancias, no similaridades, por lo que hay que tomar los primeros índices. 

In [46]:
top_similarities = similarities_lda.argsort()[:10]

df_goodreads.iloc[top_similarities]['title']

book_id
2                                 Twilight (Twilight, #1)
9284                       Redeemed (House of Night, #12)
2683                        Shadowland (The Mediator, #1)
8163           Endless Knight (The Arcana Chronicles, #2)
7372    A Court of Wings and Ruin (A Court of Thorns a...
4407                          Dreamless (Starcrossed, #2)
5371                           Reunion (The Mediator, #3)
1555                                       Opal (Lux, #3)
3815           End of Days (Penryn & the End of Days, #3)
9919                 Dream Dark (Caster Chronicles, #2.5)
Name: title, dtype: object

<div><font size=3 color=#00586D> <i class="fa fa-check-square-o" aria-hidden="true"></i></font></div>

<div class="alert alert-block alert-success" markdown=1>

__Comentarios__. _Correcto_
```python

```
</div>


En este caso las recomendaciones no son tan evidentes. Sin embargo, se aprecia que la temática es muy parecida a la de _"Twilight"_.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#00586D"></i> </font></div>